In [ ]:
#################################################################### First Part: Calculate Coordinates and Generate Feature Plots ##########################################################
setwd("china tree mapping function")

# Load necessary data
load("china_dim.RData")
load("Chinese_gam_x.rds")
load("Chinese_gam_y.rds")

##################################### The function definition below does not need modification ######################################################
ddrtree_map <- function(data) {
  library(mgcv)
  library(ggplot2)
  
  # Prepare the data
  names(data)[1] <- 'GENDER'
  names(data)[2] <- 'age'
  names(data)[3] <- 'hdl'
  names(data)[4] <- 'ldl'
  names(data)[5] <- 'tg'
  names(data)[6] <- 'hba1c'
  names(data)[7] <- 'BMI'
  names(data)[8] <- 'SBP'
  names(data)[9] <- 'DBP'
  names(data)[10] <- 'alt'
  names(data)[11] <- 'cr'
  names(data)[12] <- 'HR'
  
  print(names(data))
  
  # Data type conversion
  data[, c(2:12)] <- apply(data[, c(2:12)], 2, function(x) as.numeric(as.character(x)))
  data$GENDER <- factor(data$GENDER, levels = c('1', '2'))
  
  # Get the position prediction
  dim1 <- predict.gam(Chinese_gam_x, newdata = data)
  dim2 <- predict.gam(Chinese_gam_y, newdata = data)
  
  # Initialize new columns
  predict_data <- as.data.frame(cbind(dim1, dim2))
  predict_data$hk_test_dim1 <- NA
  predict_data$hk_test_dim2 <- NA
  
  # Reposition prediction points based on the Euclidean distance
  for (i in 1:nrow(predict_data)) {
    distance <- ((china_dim$data_dim_1 - predict_data$dim1[i])^2 + (china_dim$data_dim_2 - predict_data$dim2[i])^2)^0.5
    j <- which.min(distance)
    predict_data$hk_test_dim1[i] <- china_dim$data_dim_1[j]
    predict_data$hk_test_dim2[i] <- china_dim$data_dim_2[j]
  }
  
  # Check structure of predict_data
  print(head(predict_data))
  
  new_predict_data <- as.data.frame(cbind(predict_data, data))
  
  # Define indicators and color settings
  indicators <- c("SBP","DBP","hdl","ldl","BMI","tg","cr","alt","hba1c","HR")  # Indicator column names
  color_settings <- list(
    SBP = list(color_title = "SBP", title = "SBP (mm of Hg)", limits = c(80, 200), breaks = c(100, 120, 140, 160, 180), labels = c("100", "120", "140", "160", "180"), low = "#00FFFF", high = "red"),
    DBP = list(color_title = "DBP", title = "DBP (mm of Hg)", limits = c(40, 140), breaks = c(60, 80, 100, 120), labels = c("60", "80", "100", "120"), low = "#00FFFF", high = "red"),
    hdl = list(color_title = "HDL-C", title = "HDL-C (mmol/L)", limits = c(0, 2), breaks = c(0.5, 1, 1.5), labels = c("0.5", "1", "1.5"), low = "#00FFFF", high = "red"),
    ldl = list(color_title = "LDL-C", title = "LDL-C (mmol/L)", limits = c(0, 5), breaks = c(2, 4), labels = c("2", "4"), low = "#00FFFF", high = "red"),
    BMI = list(color_title = "BMI", title = "BMI (Kg/m2)", limits = c(20, 30), breaks = c(22, 24, 26, 28), labels = c("22", "24", "26", "28"), low = "#00FFFF", high = "red"),
    tg = list(color_title = "TRIG", title = "Triglycerides (mmol/L)", limits = c(0, 5), breaks = c(2, 4), labels = c("2", "4"), low = "#00FFFF", high = "red"),
    cr = list(color_title = "CREAT", title = "Creatinine (umol/L)", limits = c(50, 100), breaks = c(60, 70, 80, 90), labels = c("60", "70", "80", "90"), low = "#00FFFF", high = "red"),
    alt = list(color_title = "ALT", title = "ALT (IU/L)", limits = c(0, 75), breaks = c(15, 30, 45, 60), labels = c("15", "30", "45", "60"), low = "#00FFFF", high = "red"),
    hba1c = list(color_title = "HbA1c", title = "HbA1c (%)", limits = c(5, 12), breaks = c(6, 8, 10), labels = c("6", "8", "10"), low = "#00FFFF", high = "red"),
    HR = list(color_title = "HR", title = "HR (bpm)", limits = c(45, 135), breaks = c(60, 75, 90, 105, 120), labels = c("60", "75", "90", "105", "120"), low = "#00FFFF", high = "red")
  )
  
  create_plot <- function(indicator) {
    settings <- color_settings[[indicator]]
    
    # Base layer
    k <- ggplot(china_dim, aes(x = data_dim_1, y = data_dim_2)) +
      geom_point(color = 'grey') +  # Original data points in grey
      xlab('Dimension 1') +
      ylab('Dimension 2') +
      theme_minimal() +
      labs(title = settings$title) +
      theme(plot.title = element_text(size = 22, face = "bold"))
    
    # Add new layer on top of prediction points
    final_plot <- k +
      geom_point(data = new_predict_data, aes_string(x = 'hk_test_dim1', y = 'hk_test_dim2', color = indicator), size = 2) +
      scale_color_gradient(
        limits = settings$limits,
        breaks = settings$breaks,
        labels = settings$labels,
        low = settings$low,
        high = settings$high,
        oob = scales::squish
      ) +
      labs(color = settings$color_title) +
      theme(legend.text = element_text(size = 15),
            legend.title = element_text(size = 16))
    
    return(final_plot)
  }
  
  # Return the predicted data
  return(list(predict_data = predict_data, create_plot = create_plot, indicators = indicators))
}

###########################################################################
# Read data, modify the file path as necessary
data <- read.csv("sampledata.csv")

data2 <- data[, !names(data) %in% "id"] #remove col"id"

# Call ddrtree_map function to get predictions, no modifications needed
result <- ddrtree_map(data2)

# Get predicted data, no modifications needed
predicted_data <- result$predict_data
new_data <- cbind(data, predicted_data)

# Save the data, modify the file path as necessary
write.csv(new_data, "new_data.csv")

# Generate and save the plots for each indicator, modify the file path as necessary
for (indicator in result$indicators) {
  plot <- result$create_plot(indicator)
  ggsave(paste0("/Users/miaoji/Desktop/安徽省立医院数据库/", indicator, "_plot.png"), plot, width = 8, height = 6)
}

#################################################################### Second Part: Competing Risk Model to Calculate Complication Risk Probability ########################
outcome_data <- read.csv("sampleoutcome.csv")  # Load the dataset containing outcome data

final_data <- merge(outcome_data, new_data, by = "id")  # Merge with the dataset containing the calculated coordinates, ensuring accurate matching

# Load necessary packages
library(cmprsk)
library(rms)

################################ General Function: Calculate survival time and cumulative incidence probability, no need to modify #####################
calculate_survival_and_prediction <- function(final_data, outcome_name, outcome_date_col, risk_col, crr_model_covariates) {
  # Calculate survival time
  survival_time_col <- paste0("survival_time_", outcome_name)
  final_data[[survival_time_col]] <- as.numeric((as.Date(final_data[[outcome_date_col]]) - as.Date(final_data$visit_start_date)) / 365)
  
  # Calculate risk variable
  risk_col_name <- paste0("risk_", outcome_name)
  final_data[[risk_col_name]] <- ifelse(final_data[[risk_col]] == 0 & final_data$dead == 0, 0, 
                                        ifelse(final_data[[risk_col]] == 1 & final_data$dead == 0, 1, 2))
  
  # Check for missing values in survival time and status, and filter data
  valid_data <- final_data[!is.na(final_data[[survival_time_col]]) & !is.na(final_data[[risk_col_name]]), ]
  
  # Create Cox regression model (crr)
  crr_model <- crr(ftime = valid_data[[survival_time_col]],
                   fstatus = valid_data[[risk_col_name]],
                   cov1 = as.matrix(valid_data[, crr_model_covariates]),
                   cencode = 0,
                   failcode = 1)
  
  # Output model summary
  print(summary(crr_model))
  
  # Get cumulative incidence probabilities from the fitted model
  predictions <- predict(crr_model, cov1 = as.matrix(valid_data[, crr_model_covariates]))
  
  # Get cumulative incidence probability at the last time point
  last_time_point_predictions <- predictions[, ncol(predictions)]
  
  # Format prediction results
  predictions_formatted <- format(last_time_point_predictions, digits = 4)
  
  # Create result column and add prediction results to the full dataset
  probability_col_name <- paste0(outcome_name, "_probability")
  final_data[[probability_col_name]] <- NA  # Initialize the result column
  final_data[[probability_col_name]][!is.na(final_data[[survival_time_col]]) & !is.na(final_data[[risk_col_name]])] <- predictions_formatted
  
  return(final_data)
}

################## Define outcome names, corresponding date columns, and risk columns (modify based on actual data) ################
outcome_list <- list(
  list("name" = "MAFLD", "date_col" = "MAFLD_date", "risk_col" = "MAFLD"),
  list("name" = "Cirrhosis", "date_col" = "Cirrhosis_date", "risk_col" = "Cirrhosis"),
  list("name" = "MI", "date_col" = "MI_date", "risk_col" = "MI"),
  list("name" = "Stroke", "date_col" = "Stroke_date", "risk_col" = "Stroke"),
  list("name" = "Istroke", "date_col" = "Istroke_date", "risk_col" = "Istroke"),
  list("name" = "IHstroke", "date_col" = "IHstroke_date", "risk_col" = "IHstroke"),
  list("name" = "HF", "date_col" = "HF_date", "risk_col" = "HF"),
  list("name" = "HFrEF", "date_col" = "HFrEF_date", "risk_col" = "HFrEF"),
  list("name" = "HFpEF", "date_col" = "HFpEF_date", "risk_col" = "HFpEF"),
  list("name" = "CKD", "date_col" = "CKD_date", "risk_col" = "CKD"),
  list("name" = "ESRD", "date_col" = "ESRD_date", "risk_col" = "ESRD"),
  list("name" = "DR", "date_col" = "DR_date", "risk_col" = "DR")
)

# No modification needed
crr_model_covariates <- c("hk_test_dim1", "hk_test_dim2")

# Loop through each outcome to calculate survival time and cumulative incidence probability. No modification needed. #########
for (outcome in outcome_list) {
  final_data <- calculate_survival_and_prediction(
    final_data,
    outcome_name = outcome$name,
    outcome_date_col = outcome$date_col,
    risk_col = outcome$risk_col,
    crr_model_covariates = crr_model_covariates
  )
}

# View the final merged dataset
colnames(final_data)
head(final_data)

write.csv(final_data, "final_data.csv")

####################################################### Third Part: Visualization ###################################################
final_data <- read.csv("final_data.csv")  # Load the saved final_data.csv

colnames(final_data)
library(ggplot2)
library(scales)
library(purrr)

### Modify based on actual data
indicators <- c("MAFLD_probability", "Cirrhosis_probability", "MI_probability", "Stroke_probability", "Istroke_probability", "IHstroke_probability", "HF_probability", "HFrEF_probability", "HFpEF_probability", "CKD_probability", "ESRD_probability", "DR_probability")  # Get indicator column names
color_settings <- list(
  MAFLD_probability = list(color_title = "Probability of MAFLD", title = "Probability of MAFLD",  low = "#00FFFF", high = "red"),
  Cirrhosis_probability = list(color_title = "Probability of Cirrhosis", title = "Probability of Cirrhosis", low = "#00FFFF", high = "red"),
  MI_probability = list(color_title = "Probability of MI", title = "Probability of MI",  low = "#00FFFF", high = "red"),
  Stroke_probability = list(color_title = "Probability of Stroke", title = "Probability of Stroke",  low = "#00FFFF", high = "red"),
  Istroke_probability = list(color_title = "Probability of Istroke", title = "Probability of Istroke", low = "#00FFFF", high = "red"),
  IHstroke_probability = list(color_title = "Probability of IHstroke", title = "Probability of IHstroke",  low = "#00FFFF", high = "red"),
  HF_probability = list(color_title = "Probability of HF", title = "Probability of HF",  low = "#00FFFF", high = "red"),
  HFrEF_probability = list(color_title = "Probability of HFrEF", title = "Probability of HFrEF", low = "#00FFFF", high = "red"),
  HFpEF_probability = list(color_title = "Probability of HFpEF", title = "Probability of HFpEF", low = "#00FFFF", high = "red"),
  CKD_probability = list(color_title = "Probability of CKD", title = "Probability of CKD", low = "#00FFFF", high = "red"),
  ESRD_probability = list(color_title = "Probability of ESRD", title = "Probability of ESRD", low = "#00FFFF", high = "red"),
  DR_probability = list(color_title = "Probability of DR", title = "Probability of DR",  low = "#00FFFF", high = "red")
)

############################### Define function, no need to modify #####################################
create_plot <- function(indicator) {
  settings <- color_settings[[indicator]]
  
  # Ensure the indicator column is numeric
  final_data[[indicator]] <- as.numeric(final_data[[indicator]])
  
  # Base plot
  k <- ggplot(china_dim, aes(x = data_dim_1, y = data_dim_2)) +
    geom_point(color = 'grey') +  # Original data points in grey
    xlab('Dimension 1') +
    ylab('Dimension 2') +
    theme_minimal() +
    labs(title = settings$title) +
    theme(plot.title = element_text(size = 22, face = "bold"))
  
  # Overlay new layer with prediction points
  final_plot <- k +
    geom_point(data = final_data, aes(x = hk_test_dim1, y = hk_test_dim2, color = !!sym(indicator)), size = 2) + 
    scale_color_gradient(
      low = settings$low,
      high = settings$high
    ) +
    labs(color = settings$color_title) +
    theme(legend.text = element_text(size = 15),
          legend.title = element_text(size = 16))
  
  return(final_plot)
}

#################################################################### Plotting #########
plots <- map(indicators, create_plot)
folder_path <- "/Users/miaoji/Desktop/安徽省立医院数据库/"  # Set the path to save the images

#########no need to modify #####################################
save_plot <- function(plot, indicator) {
  file.name <- paste0("plot_outcome_", indicator, ".png")
  file_path <- file.path(folder_path, file.name)
  ggsave(filename = file_path, plot = plot, width = 8, height = 6)
  cat("Plot saved as:", file_path, "\n")
}

walk2(plots, indicators, save_plot)
